In [1]:
import pandas as pd
df = pd.read_csv('df_no_images.csv')

In [2]:
df

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,...,Language,Category,city,state,country,Category_frequency,Language_frequency,Country_frequency,City_frequency,Publisher_frequency
0,835924,88984,"sydney, nsw, australia",34.7439,0395717566,0,the best american essays 1996 (serial),Geoffrey C. Ward,1996.0,Mariner Books,...,en,literary collections,sydney,nsw,australia,1289,591097,18197,3944.0,929
1,16726,259066,"oakley, california, usa",56.0000,0316776963,9,me talk pretty one day,David Sedaris,2001.0,Back Bay Books,...,9,9,oakley,california,usa,386696,379929,744984,38.0,3785
2,940754,271705,"atlanta, georgia, usa",25.0000,0192834371,0,the communist manifesto (oxford world's classics),Karl Marx,1998.0,Oxford University Press,...,en,philosophy,atlanta,georgia,usa,1719,591097,744984,1915.0,2563
3,965187,186570,"rantoul, illinois, usa",28.0000,0380758636,0,hint of rapture,Miriam Minger,1990.0,Harper Mass Market Paperbacks (Mm),...,9,9,rantoul,illinois,usa,386696,379929,744984,592.0,4298
4,277519,255651,"urbana, illinois, usa",34.7439,0679451145,0,the island of the colorblind,Oliver W. Sacks,1997.0,Random House Inc,...,en,science,urbana,illinois,usa,2069,591097,744984,1025.0,6291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99964,425513,193898,"paradise, pennsylvania, usa",34.7439,0380726246,0,prayers for the dead : a peter decker/rina laz...,Faye Kellerman,1997.0,Avon,...,en,fiction,paradise,pennsylvania,usa,368034,591097,744984,233.0,16635
99965,839275,257198,"edmonton, alberta, canada",34.0000,0061054372,7,the unauthorized history of trek (harper prism),James Van Hise,1995.0,Harper Mass Market Paperbacks (Mm),...,en,performing arts,edmonton,alberta,canada,1483,591097,92571,4036.0,4298
99966,759782,105374,"montreal, quebec, canada",25.0000,0192876791,0,paul (past masters),E. P. Sanders,1991.0,Oxford University Press,...,en,history,montreal,quebec,canada,7855,591097,92571,2052.0,2563
99967,727870,189334,"ottawa, ontario, canada",49.0000,0345303075,10,white gold wielder (the second chronicles of t...,Stephen R. Donaldson,1983.0,Ballantine Books,...,9,9,ottawa,ontario,canada,386696,379929,92571,8142.0,33161


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Kitap başlıkları, yazarlar ve kategoriler için TF-IDF vektörizasyonu
df['combined_features'] = df['book_title'] + ' ' + df['book_author'] + ' ' + df['Category']

# TF-IDF vektör matrisi
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

print(tfidf_matrix.shape)  # Vektör boyutunu kontrol edin

(99969, 45614)


In [4]:
def filter_books_by_user_info(age, city, country, language, df):
    # Kullanıcının yaş, şehir, ülke ve dil bilgilerine göre kitapları filtreleme
    filtered_books = df[(df['age'] <= age + 5) & 
                        (df['age'] >= age - 5) & 
                        (df['city'] == city) & 
                        (df['country'] == country) &
                        (df['Language'] == language)]
    
    return filtered_books

# Örnek kullanım
age = 30
city = 'New York'
country = 'United States'
language = 'en'

filtered_books = filter_books_by_user_info(age, city, country, language, df)
print(filtered_books.head())  # Filtrelenmiş kitapları görüntüleyin

Empty DataFrame
Columns: [Unnamed: 0, user_id, location, age, isbn, rating, book_title, book_author, year_of_publication, publisher, Summary, Language, Category, city, state, country, Category_frequency, Language_frequency, Country_frequency, City_frequency, Publisher_frequency, combined_features]
Index: []

[0 rows x 22 columns]


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Kullanıcı-kitap matrisi oluşturma
user_book_matrix = df.pivot_table(index='user_id', columns='book_title', values='rating').fillna(0)

# Kullanıcı benzerlik matrisini hesaplama
user_similarity = cosine_similarity(user_book_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_book_matrix.index, columns=user_book_matrix.index)

def get_user_recommendations(user_id, user_book_matrix, user_similarity_df):
    # Benzer kullanıcıları bul
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:6]
    
    # Benzer kullanıcıların ortalama puanlarını hesapla
    similar_users_ratings = user_book_matrix.loc[similar_users].mean()
    
    # Kullanıcının henüz puan vermediği kitapları öner
    user_ratings = user_book_matrix.loc[user_id]
    recommendations = similar_users_ratings[user_ratings == 0].sort_values(ascending=False)
    
    return recommendations.index.tolist()

# Örnek kullanım
user_id = 88984
user_recommendations = get_user_recommendations(user_id, user_book_matrix, user_similarity_df)
print(user_recommendations[:10])  # İlk 10 öneriyi görüntüleyin

['a prayer for owen meany', 'blue shoe', '!yo!', 'soledad: a novel', 'soldier of light', 'soldier of the mist', "soldier's heart", "soldier's heart : being the story of the enlistment and due service of the boy charley goddard in the first minnesota volunteers", 'soldiers of fortune', 'soldiers of paradise']


In [6]:
def hybrid_recommendations(user_id, book_title, age, city, country, language, tfidf_matrix, df, user_book_matrix, user_similarity_df):
    # Content-Based Filtering sonuçları
    cbf_recommendations = get_recommendations(book_title, tfidf_matrix, df)
    
    # Kullanıcı bilgileriyle Content-Based sonuçlarını filtreleme
    filtered_cbf_recommendations = df[(df['book_title'].isin(cbf_recommendations)) &
                                      (df['age'] <= age + 5) & (df['age'] >= age - 5) &
                                      (df['city'] == city) & (df['country'] == country) &
                                      (df['Language'] == language)]
    
    # Collaborative Filtering sonuçları
    cf_recommendations = get_user_recommendations(user_id, user_book_matrix, user_similarity_df)
    
    # Sonuçları birleştirme
    combined_recommendations = list(set(filtered_cbf_recommendations['book_title']) | set(cf_recommendations))
    
    return combined_recommendations

# Örnek kullanım
hybrid_recs = hybrid_recommendations(user_id=88984, book_title='the best american essays 1996 (serial)', 
                                     age=30, city='New York', country='United States', language='en', 
                                     tfidf_matrix=tfidf_matrix, df=df, user_book_matrix=user_book_matrix, 
                                     user_similarity_df=user_similarity_df)
print(hybrid_recs[:10])  # İlk 10 öneriyi görüntüleyin

NameError: name 'get_recommendations' is not defined

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_recommendations(book_title, tfidf_matrix, df):
    # Kitabın indeksini bul
    idx = df[df['book_title'] == book_title].index[0]
    
    # Kitaplar arasındaki benzerliği hesapla
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    
    # Benzerlik sırasına göre kitapları sırala
    indices = np.argsort(cosine_sim)[::-1]
    
    # Kitap başlıklarını döndür
    return df['book_title'].iloc[indices].values

# Örnek kullanım
recommendations = get_recommendations('the best american essays 1996 (serial)', tfidf_matrix, df)
print(recommendations[:10])  # İlk 10 öneriyi görüntüleyin

['the best american essays 1996 (serial)' 'the best american essays, 1992'
 'how to be alone: essays'
 'the best american essays 1994 (the best american essays)'
 'the best american short stories 1996: selected from u.s. and canadian magazines (serial)'
 'small wonder: essays' 'small wonder : essays' 'small wonder: essays'
 'small wonder: essays' 'essays of e. b. white (perennial classics)']


In [8]:
# Hybrid sistemi tekrar çalıştırıyoruz
hybrid_recs = hybrid_recommendations(user_id=88984, book_title='the best american essays 1996 (serial)', 
                                     age=30, city='New York', country='United States', language='en', 
                                     tfidf_matrix=tfidf_matrix, df=df, user_book_matrix=user_book_matrix, 
                                     user_similarity_df=user_similarity_df)
print(hybrid_recs[:10])  # İlk 10 öneriyi görüntüleyin

["norman vincent peale's treasury of joy and enthusiasm", 'cassidy harte and the comeback kid (outlaw hartes) (silhouette intimate moments, no. 1144)', 'the mozart season (point)', 'andorra', 'mark mcgwire and chipper jones', 'murder most cozy', 'alphabets and ornaments', 'love me', 'true devotion (love inspired, no. 241)', 'the mystery at number seven, rue petite']


In [9]:
import ipywidgets as widgets
from IPython.display import display

# Kullanıcıdan alınacak bilgiler için widget'lar oluşturma
age_widget = widgets.IntText(description='Age:', placeholder='Enter your age')
city_widget = widgets.Text(description='City:', placeholder='Enter your city')
country_widget = widgets.Text(description='Country:', placeholder='Enter your country')
language_widget = widgets.Text(description='Language:', placeholder='Enter your preferred language')
book_title_widget = widgets.Text(description='Book Title:', placeholder='Enter a book title')
user_id_widget = widgets.IntText(description='User ID:', placeholder='Enter your user ID')

recommend_button = widgets.Button(description='Get Recommendations')

# Öneri butonu için fonksiyon
def on_button_click(b):
    user_id = user_id_widget.value
    age = age_widget.value
    city = city_widget.value
    country = country_widget.value
    language = language_widget.value
    book_title = book_title_widget.value
    
    recommendations = hybrid_recommendations(user_id=user_id, book_title=book_title, 
                                             age=age, city=city, country=country, 
                                             language=language, tfidf_matrix=tfidf_matrix, 
                                             df=df, user_book_matrix=user_book_matrix, 
                                             user_similarity_df=user_similarity_df)
    
    if recommendations:
        print("Top 10 Recommendations:")
        for i, rec in enumerate(recommendations[:10]):
            print(f"{i + 1}: {rec}")
    else:
        print("No recommendations found.")

# Düğmeye tıklama olayını bağlayın
recommend_button.on_click(on_button_click)

# Widget'ları ekranda gösterme
display(user_id_widget, book_title_widget, age_widget, city_widget, country_widget, language_widget, recommend_button)

IntText(value=0, description='User ID:')

Text(value='', description='Book Title:', placeholder='Enter a book title')

IntText(value=0, description='Age:')

Text(value='', description='City:', placeholder='Enter your city')

Text(value='', description='Country:', placeholder='Enter your country')

Text(value='', description='Language:', placeholder='Enter your preferred language')

Button(description='Get Recommendations', style=ButtonStyle())

Top 10 Recommendations:
1: norman vincent peale's treasury of joy and enthusiasm
2: cassidy harte and the comeback kid (outlaw hartes) (silhouette intimate moments, no. 1144)
3: the mozart season (point)
4: andorra
5: mark mcgwire and chipper jones
6: murder most cozy
7: alphabets and ornaments
8: love me
9: true devotion (love inspired, no. 241)
10: the mystery at number seven, rue petite


In [10]:
df

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,...,Category,city,state,country,Category_frequency,Language_frequency,Country_frequency,City_frequency,Publisher_frequency,combined_features
0,835924,88984,"sydney, nsw, australia",34.7439,0395717566,0,the best american essays 1996 (serial),Geoffrey C. Ward,1996.0,Mariner Books,...,literary collections,sydney,nsw,australia,1289,591097,18197,3944.0,929,the best american essays 1996 (serial) Geoffre...
1,16726,259066,"oakley, california, usa",56.0000,0316776963,9,me talk pretty one day,David Sedaris,2001.0,Back Bay Books,...,9,oakley,california,usa,386696,379929,744984,38.0,3785,me talk pretty one day David Sedaris 9
2,940754,271705,"atlanta, georgia, usa",25.0000,0192834371,0,the communist manifesto (oxford world's classics),Karl Marx,1998.0,Oxford University Press,...,philosophy,atlanta,georgia,usa,1719,591097,744984,1915.0,2563,the communist manifesto (oxford world's classi...
3,965187,186570,"rantoul, illinois, usa",28.0000,0380758636,0,hint of rapture,Miriam Minger,1990.0,Harper Mass Market Paperbacks (Mm),...,9,rantoul,illinois,usa,386696,379929,744984,592.0,4298,hint of rapture Miriam Minger 9
4,277519,255651,"urbana, illinois, usa",34.7439,0679451145,0,the island of the colorblind,Oliver W. Sacks,1997.0,Random House Inc,...,science,urbana,illinois,usa,2069,591097,744984,1025.0,6291,the island of the colorblind Oliver W. Sacks s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99964,425513,193898,"paradise, pennsylvania, usa",34.7439,0380726246,0,prayers for the dead : a peter decker/rina laz...,Faye Kellerman,1997.0,Avon,...,fiction,paradise,pennsylvania,usa,368034,591097,744984,233.0,16635,prayers for the dead : a peter decker/rina laz...
99965,839275,257198,"edmonton, alberta, canada",34.0000,0061054372,7,the unauthorized history of trek (harper prism),James Van Hise,1995.0,Harper Mass Market Paperbacks (Mm),...,performing arts,edmonton,alberta,canada,1483,591097,92571,4036.0,4298,the unauthorized history of trek (harper prism...
99966,759782,105374,"montreal, quebec, canada",25.0000,0192876791,0,paul (past masters),E. P. Sanders,1991.0,Oxford University Press,...,history,montreal,quebec,canada,7855,591097,92571,2052.0,2563,paul (past masters) E. P. Sanders history
99967,727870,189334,"ottawa, ontario, canada",49.0000,0345303075,10,white gold wielder (the second chronicles of t...,Stephen R. Donaldson,1983.0,Ballantine Books,...,9,ottawa,ontario,canada,386696,379929,92571,8142.0,33161,white gold wielder (the second chronicles of t...


In [ ]:
# created by azad halhalli
